In [1]:
#!/usr/bin/env python3
import sevenbridges as sbg
import sys
import re
import os
import concurrent.futures
# import pdb
# from requests import request
# from datetime import datetime
config = sbg.Config(profile='cavatica')
api = sbg.Api(config=config)

### Get all completed alignment tasks

In [6]:
def check_task(task):
    tname = task.name.split('-')
    try:
        if tname[0] == 'alignment':
            bs_id = task.inputs['biospecimen_name']
            if tname[1] != bs_id:
                sys.stderr.write('ERROR: Mismatch bs id in task name and biospecimen name for ' + task.id + ' ' + task.name + ' ' + bs_id + '\n')

            return task.id + '\t' + task.name + '\t' + bs_id + '\t' + task.inputs['input_reads'].name + '\t' + task.inputs['input_reads'].id + '\n'
        else:
            return 'poop'
    except Exception as e:
        sys.stderr.write(str(e) + '\n')
        sys.stderr.write('Could not process task' + task.name + '\n')
        sys.stderr.flush()
        return 'poop'

In [9]:
uname = 'kfdrc-harmonization/sd-preasa7s'
pname = 'seidman_fy15'

tasks = api.tasks.query(project=uname, status='COMPLETED').all()
out_dir = '/Users/brownm28/Documents/2019-Jan-30_reharmonization/seidman_fy15/'
out = open(out_dir + pname + '_all_align_tasks.txt', 'w')
out.write('Original task ID\tTask name\tBS ID\tinput bam\tinput file ID\n')
x = 1
n = 100
with concurrent.futures.ThreadPoolExecutor(8) as executor:
    results = {executor.submit(check_task, task): task for task in tasks}
    for result in concurrent.futures.as_completed(results):
        if x % n == 0:
            sys.stderr.write('Processed ' + str(x) + ' tasks for project ' + pname + '\n')
            sys.stderr.flush()
        x += 1
        if result.result() != 'poop':
            out.write(result.result())
out.close()

Processed 100 tasks for project seidman_fy15
Processed 200 tasks for project seidman_fy15
Processed 300 tasks for project seidman_fy15
Processed 400 tasks for project seidman_fy15
Processed 500 tasks for project seidman_fy15
Processed 600 tasks for project seidman_fy15
Processed 700 tasks for project seidman_fy15
Processed 800 tasks for project seidman_fy15
ERROR: Mismatch bs id in task name and biospecimen name for 355279bb-1fb6-4a14-888a-4b65f41c1108 alignment-BS_JA331RWK_QcReun2 BS_JA331RWK
Processed 900 tasks for project seidman_fy15
